<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/regW_gpt_3_5_turbo_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://www.federalreserve.gov/supervisionreg/legalinterpretations/reg-w-frequently-asked-questions.htm
- https://www.federalreserve.gov/supervisionreg/legalinterpretations/legalinterpretations.htm
- https://www.federalreserve.gov/supervisionreg/topics/regulation_w.htm
- https://www.federalreserve.gov/supervisionreg/reglisting.htm#W
- https://www.ecfr.gov/current/title-12/chapter-II/subchapter-A/part-223
- https://www.govinfo.gov/content/pkg/CFR-2022-title12-vol3/pdf/CFR-2022-title12-vol3-part223.pdf


In [1]:
!mkdir -p DATA/RegW;curl https://www.govinfo.gov/content/pkg/CFR-2022-title12-vol3/pdf/CFR-2022-title12-vol3-part223.pdf -o DATA/RegW/CFR-2022-title12-vol3-part223.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  275k  100  275k    0     0   727k      0 --:--:-- --:--:-- --:--:--  727k


In [2]:
%pip install -qq llama-index openai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.0/580.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.9 MB/s eta 0:00:00


In [3]:
import llama_index
from llama_index import ListIndex, VectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext
from llama_index.response.notebook_utils import display_response
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import logging, sys, os, json
from pathlib import Path
import openai
from typing import List, Tuple, Dict, Union

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
cwd: Path = Path.cwd()
data_dir: Path = cwd / "DATA"
regw_dir: Path  = data_dir / "RegW"
documents: List = SimpleDirectoryReader(regw_dir).load_data()

In [8]:
def set_open_ai_key(env_path: Union[Path, str] = None) -> Tuple[bool, str]:
  import json, os
  from pathlib import Path

  openai.api_key = None #clear previous key if exists
  env_path: Path = Path(env_path).absolute() # in case env_path is passed as a str
  if not env_path.is_file(): err: str = f"File:{str(env_path)} does not exist."
  else:
    try:
      with open(env_path, "r") as f:
          env_vars = json.load(f)
      os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
      openai.api_key = os.environ["OPENAI_API_KEY"]
      openai.Model.list() #test a random command on the openai API
      err = None
    except Exception as e:
      err = json.dumps({"error":e})

  logger.info(err)
  return (True, "ok") if not err else (False, err)

In [9]:
cwd: Path = Path.cwd()
env_path: Path = cwd/ "openai.env";
print(set_open_ai_key(env_path)) # Tuple[bool, str]

(True, 'ok')


In [10]:
llm: llama_index.llms.openai.OpenAI = OpenAI(temperature=0,
             model="gpt-3.5-turbo",
             max_tokens=500,
             max_retries = 5)

service_context: llama_index.indices.service_context.ServiceContext = ServiceContext.from_defaults(llm=llm)

In [11]:
davinci_index: llama_index.indices.vector_store.base.VectorStoreIndex = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [12]:
query_engine: llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine = davinci_index.as_query_engine(response_mode="tree_summarize")
q: str = "what is the scope of Regulation W?"
response: llama_index.response.schema.Response = query_engine.query(q)
display_response(response)

**`Final Response:`** The scope of Regulation W includes member banks, which refers to national banks, State banks, trust companies, or other institutions that are members of the Federal Reserve System. Additionally, insured State non-member banks and insured savings institutions are also subject to sections 23A and 23B of Regulation W.

In [13]:
q = "what is the purpose of regulation W?"
display_response(query_engine.query(q))

**`Final Response:`** Based on the provided context information, the purpose of Regulation W is to implement section 7(f) of the Securities Exchange Act of 1934, which requires that credit obtained within or outside the United States complies with the limitations of the Board's Margin Regulations T and U. Regulation W applies the Board's margin regulations to borrowers who obtain credit outside the United States to purchase or carry United States securities, or within the United States to purchase or carry any securities. The regulation establishes certain exemptions for borrowers who meet specific criteria.

In [16]:
q = "what entity issued regulation W? Under what authority was it issued?"
display_response(query_engine.query(q))

KeyboardInterrupt: ignored

In [15]:
q = """refer to subpart A to answer the following questions.
Question#1:::what entity issued regulation W?
Question#2:::Under what authority was it issued?"""
display_response(query_engine.query(q))

**`Final Response:`** Question#1: The Board of Governors of the Federal Reserve System (Board) issued Regulation W.

Question#2: Regulation W was issued under the authority of sections 23A(f) and 23B(e) of the Federal Reserve Act (FRA) (12 U.S.C. 371c(f), 371c–1(e)), section 11 of the Home Owners’ Loan Act (12 U.S.C. 1468), and section 312(b)(2)(A) of the Dodd-Frank Wall Street Reform and Consumer Protection Act (12 U.S.C. 5412).